### 기 - 컬럼명 뜻 몰라서 적음
SIG_CD = 시군구코드 / RDS_MAN_NO = 도로구간일련번호 / RN = 도로명 / RN_CD = 도로명번호

In [1]:
import pandas as pd
import numpy as np

all_coords = pd.read_csv("1_3_COORDS_IN_YS.csv",encoding = 'ms949')
lights = pd.read_csv("3_load_with_light.csv",encoding = 'ms949')

all_coords.drop(columns = "Unnamed: 0" , inplace = True)
all_coords.head(2)

lights.drop(columns = "Unnamed: 0" , inplace = True)
#lights = lights[ lights['분류'] == '도로']
lights
lights.head()

,SIG_CD,RDS_MAN_NO,RDS_MAN_NO2,RN,RN_CD,LAT,LNG,분류,밝기
0,11680,12,0,역삼로,3122008,37.493030,127.029968,도로,0
1,11680,12,1,역삼로,3122008,37.493107,127.030238,도로,0
2,11680,12,2,역삼로,3122008,37.493325,127.030953,도로,20
3,11680,12,3,역삼로,3122008,37.493571,127.031758,도로,0
4,11680,12,4,역삼로,3122008,37.493818,127.032570,도로,0


In [2]:
test = lights.sort_values(by = [ 'RN_CD' , 'RDS_MAN_NO', 'RDS_MAN_NO2' ])
test.reset_index(inplace = True,drop = True)
test

np.mean( np.array(test.loc[0, '밝기'],   test.loc[1, '밝기'] ) )   # 0.0

0.0

In [3]:
link_df = pd.DataFrame(columns = [ 'LINK_NO','RDS_MAN_NO', 'RN', 'RN_CD', 'SP','EP', 'LUX']  )
n = 0
#링크 정보 저장
for i in range(1, len(test)-1):
    if test.loc[i, 'RDS_MAN_NO'] == test.loc[i+1, 'RDS_MAN_NO'] :
        tmp = { 'LINK_NO' : 'LINK_%d' % n 
              ,'RDS_MAN_NO' : test.loc[i,'RDS_MAN_NO']
              , 'RN' : test.loc[i,'RN']
              , 'RN_CD' : test.loc[i,'RN_CD']
              , 'SP' : [test.loc[i, 'LAT'], test.loc[i, 'LNG']]
              , 'EP' : [test.loc[i+1, 'LAT'],test.loc[i+1, 'LNG']]
              , 'LUX' : np.mean( np.array(test.loc[i, '밝기'], test.loc[i+1, '밝기'] ) )

              }
        link_df = link_df.append( link_df.from_dict(tmp, orient = 'index').T) 
        n += 1


In [4]:
link_df = link_df.loc[:,['LINK_NO','RDS_MAN_NO', 'RN', 'RN_CD','SP','EP','LUX']].reset_index(drop = True)
link_df.to_csv("links_with_lux.csv", mode='w', encoding='ms949') # edge(=link) list
link_df
link_df.head()

,LINK_NO,RDS_MAN_NO,RN,RN_CD,SP,EP,LUX
0,LINK_0,12,역삼로,3122008,"[37.493107479897255, 127.0302380152928]","[37.49332549900935, 127.03095311761676]",0
1,LINK_1,12,역삼로,3122008,"[37.49332549900935, 127.03095311761676]","[37.49357073712253, 127.0317577936122]",20
2,LINK_2,12,역삼로,3122008,"[37.49357073712253, 127.0317577936122]","[37.493818301322776, 127.03257011999749]",0
3,LINK_3,12,역삼로,3122008,"[37.493818301322776, 127.03257011999749]","[37.49387965126336, 127.03277182263642]",0
4,LINK_4,12,역삼로,3122008,"[37.49387965126336, 127.03277182263642]","[37.493917325330926, 127.03289544556885]",20


In [5]:
import os
import folium
import networkx as nx

#필요한 열만 가져오기
nodes = test[['RN_CD','RN', 'RDS_MAN_NO','LAT','LNG']]
links = link_df[['SP','EP','LUX']]

std_point = (nodes.loc[0,'LAT'], nodes.loc[0,'LNG'])
map_osm = folium.Map(location=std_point, zoom_start=17) 


#### 기석 잘 몰라서 적어둠

kw = {'opacity': 0.5, 'weight': LUX/10}

 iterrows()  -> 속도가 빨라서 쓰는듯
 
 opacity : 투명도
 
 **kwargs  (keyword arguments)** : 파라미터 명을 같이 보낼 수 있다. kwargs는 딕셔너리() 형태로 전달된다


In [7]:

for ix, row in links.iterrows():
    start =  (row['SP']) # 위도, 경도 튜플
    end =   (row['EP'])
    kw = { 'opacity': 1, 'weight': row['LUX']/10 }
    
    #print(kw['weight']) #lux 범위가 0~16사이로 나옴.  우선 3등분 함     
    if kw['weight'] < 5 :
        folium.PolyLine(
            locations=[start, end],
            color='#FF00CC',      # 노락색(yellow)은 잘 안보임, 주황색(orange)도 그닥인듯
            line_cap='round',
            **kw,
        ).add_to(map_osm)
    elif kw['weight'] < 10 :
        folium.PolyLine(
            locations=[start, end],
            color='#FF6633',
            line_cap='round',
            **kw,
        ).add_to(map_osm)
    else :
        folium.PolyLine(
            locations=[start, end],
            color='#FF0000',
            line_cap='round',
            **kw,
        ).add_to(map_osm)
            
# it takes some time.....
#map_osm